In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [2]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "wiki-research-l/2020-July"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [3]:
len(mail_df)

37

In [4]:
from preProcess import cleaningText
s_list = []
for i,content in mail_df['Content'].items():
    sentences = cleaningText(content)
    s_list.append(sentences)
mail_df['Cleaned_Content'] = s_list

In [5]:
#　センテンスのリストを作る
sentence_list = []
for content in mail_df['Cleaned_Content']:
    sentences = tokenize.sent_tokenize(content)
    for sentence in sentences:
        sentence_list.append(sentence)

In [6]:
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [7]:
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
            rows.append(row)

In [8]:
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])
len(kb_df)

853

In [9]:
from db import connect
engine = connect()
kb_df.to_sql(name='kb-wiki-reseach-l',con=engine,if_exists='replace',index=None)